In [ ]:
%matplotlib inline
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
import numpy as np
import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_isosurface
from metpy.constants import g
import xarray as xr


In [ ]:
def getdata(year='1999',month='6', day='19', hour='12'):
    
    # Reduce lat/lon to get rid of missing values
    iymin = 5
    ixmin = 5
    iymax = 270
    ixmax = 335
    
    dt = datetime(int(year),int(month),int(day),int(hour))
    
    # Grab Pressure Level Data
    air_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                               f'NARR/pressure/air.{dt:%Y%m}.nc').metpy.parse_cf()
    uwnd_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/uwnd.{dt:%Y%m}.nc').metpy.parse_cf()
    vwnd_data = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                f'NARR/pressure/vwnd.{dt:%Y%m}.nc').metpy.parse_cf()
        
    vtimes = air_data.time.sel(time=dt).values.astype('datetime64[ms]').astype('O')
      
    data = {}
    data['airtemp'] = air_data.air.metpy.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax))
    data['uwnd'] = uwnd_data.uwnd.metpy.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax))
    data['vwnd'] = vwnd_data.vwnd.metpy.sel(time=dt).isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax))
    data['time'] = vtimes
    data['level'] = air_data.level.values
    data['lat'] = air_data.lat.isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).values
    data['lon'] = air_data.lon.isel(y=slice(iymin,iymax), x=slice(ixmin, ixmax)).values
    
    return data

In [ ]:
def plot(level='250'):
    data = x.widget.result
    year = x.widget.kwargs['year']
    month = x.widget.kwargs['month']
    day = x.widget.kwargs['day']
    hour = x.widget.kwargs['hour']
        
    dt = datetime(int(year),int(month),int(day),int(hour))
        
    # Pull out the lat and lon data
    lat = data['lat'][:]
    lon = data['lon'][:]
    pres_levs = data['level'][::-1] * units.hPa
    pres_levs3 = pres_levs[:, None, None]
    ip850 = np.where(pres_levs.m == 850)[0][0]
    ip925 = np.where(pres_levs.m == 925)[0][0]
    
    if (level != 'DT'):
        lev = np.where(pres_levs.m == level)[0][0]
    
    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])
    
    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs,lon,lat)
    tlon = tlatslons[:,:,0]
    tlat = tlatslons[:,:,1]
    
    temp_var = mpcalc.smooth_n_point(data['airtemp'][::-1,:,:], 9, 10)
    uwnd_var = mpcalc.smooth_n_point(data['uwnd'][::-1,:,:], 9, 10)
    vwnd_var = mpcalc.smooth_n_point(data['vwnd'][::-1,:,:], 9, 10)

    thta_var = mpcalc.potential_temperature(pres_levs3, temp_var)
    
    relvor = mpcalc.vorticity(uwnd_var, vwnd_var)

    epv = mpcalc.potential_vorticity_baroclinic(thta_var, pres_levs3, uwnd_var, vwnd_var)
    
    if (level != 'DT'):
        wspd = mpcalc.wind_speed(uwnd_var[lev], vwnd_var[lev]).metpy.convert_units('kt')
        div = mpcalc.divergence(uwnd_var[lev], vwnd_var[lev])
    else:
        thta_DT = interpolate_to_isosurface(epv.values*1e6, thta_var.values, 2)
        uwnd_DT = interpolate_to_isosurface(epv.values*1e6, uwnd_var.values, 2)
        vwnd_DT = interpolate_to_isosurface(epv.values*1e6, vwnd_var.values, 2)
        relvor_925850 = np.average(relvor[[ip925,ip850],:,:], axis=0)
    
    fig = plt.figure(figsize=(16, 14))
    ax = fig.add_subplot(111, projection=plotcrs)
    ax.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())

    # Add state boundaries to plot
    ax.add_feature(cfeature.STATES.with_scale('50m'))
    
    # Add country borders to plot
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    
    # Contour based on variable chosen
    if (level != 'DT'):
        clevPV = np.arange(1, 20, 1)
        cs = ax.contour(tlon, tlat, epv[lev]*1e6, clevPV, linewidth=2, colors='black')
        plt.clabel(cs,inline=True,fmt='%d')
        cs2 = ax.contour(tlon, tlat, div*1e5, range(1, 50, 3), colors='k', linestyles='dashed', alpha=0.6)
        plt.clabel(cs2, fmt='%i', rightside_up=True)
        cf = ax.contourf(tlon, tlat, wspd, np.arange(50,230,20), cmap='BuPu', extend='max')
        cb = fig.colorbar(cf, orientation='horizontal', ax=ax, pad=0, aspect=50, extendrect='True')
        plt.title(r''+str(level)+'-hPa PV (hPa), Wind Speed (kts), and Divergence ($10^5$ $s^{-1}$)',
                  loc='left')
    else:
        clev_thta = np.arange(258,427,6)
        cs = ax.contour(tlon, tlat, mpcalc.smooth_n_point(thta_DT, 9, 2), clev_thta, colors='k', linestyles='dotted', alpha=0.5)
        cs2 = ax.contour(tlon, tlat, relvor_925850*1e4, np.arange(0.5,30,.5), colors='k', alpha=0.85)
        cf = ax.contourf(tlon, tlat, thta_DT, clev_thta, cmap='coolwarm', extend='both')
        cb = fig.colorbar(cf, orientation='horizontal',ax=ax,pad=0,aspect=50,extendrect='True')
        plt.title(r'Dynamic Tropopause Pot. Temp. (K), Wind Barbs (kt), 925-850-hPa Rel. Vort ($10^4$ $s^{-1}$)',
                  loc='left')
        
        # Plot Wind Barbs
        # Transform Vectors and plot wind barbs.
        wind_slice = [slice(None, None, 7), slice(None, None, 7)]
        ax.barbs(lon[wind_slice], lat[wind_slice],
                 uwnd_DT[wind_slice]*units('m/s').to('kts').m, vwnd_DT[wind_slice]*units('m/s').to('kts').m,
                 length=6, pivot='middle', alpha=0.6, transform=datacrs)

    
    plt.title('VALID: {}'.format(data['time']), loc='right')
    plt.savefig(f'PV_{level}_{dt:%Y%m%d_%H}.png', bbox_inches='tight', dpi=150)
    plt.show()

In [ ]:
year_widget = widgets.Dropdown(
    options=list(range(1979, datetime.utcnow().year+1)),
    description='Year', alignment='center')

month_widget = widgets.Dropdown(
    options=list(range(1,13)),
    description='Month', alignment='center')

day_widget = widgets.Dropdown(description='Day', options=list(range(1,32)))
hour_widget = widgets.Dropdown(description='Hour', options=[0,3,6,9,12,15,18,21])

levels = [150, 200, 250, 300, 350, 400, 'DT']
#print(levels)
#level_widget = widgets.BoundedIntText(value=300,min=100,max=400,step=1,
#                                      description='Level: ',disabled=False)
level_widget = widgets.Dropdown(description='Level', options=levels)

In [ ]:
x = widgets.interact_manual(getdata,year=year_widget,month=month_widget, day=day_widget, hour=hour_widget)

In [ ]:
print('Select the desired PV Level or the DT')
p = widgets.interact_manual(plot, level=level_widget)